In [1]:
import pandas as pd
import numpy as np

# Read taxonomy table

In [2]:
df_tax = pd.read_csv('taxonomy.tsv', sep='\t')
taxonomy = []
for idx in df_tax.index:
    taxon = df_tax.loc[idx,'Taxon']
    taxon = taxon.split(';')
    res = [df_tax.loc[idx,'Feature ID']]
    lowest_classified_taxon = 'unclassified'
    for i in np.arange(7):
        if i < len(taxon):
            res.append(taxon[i])
            if 'unclassified' not in taxon[i] and 'uncultured' not in taxon[i] and 'unidentified' not in taxon[i]:
                lowest_classified_taxon = taxon[i]
        else:
            res.append('unclassified')
    res.append(lowest_classified_taxon)
    taxonomy.append(res)
    
df_tax = pd.merge(df_tax, pd.DataFrame(taxonomy, columns=['Feature ID','Kingdom','Phylum','Class','Order','Family','Genus','Species','LowestClassifiedTaxon']), left_on='Feature ID', right_on='Feature ID', how='left')
df_tax = df_tax[['Feature ID','Confidence','Kingdom','Phylum','Class','Order','Family','Genus','Species','LowestClassifiedTaxon']]
df_tax = df_tax.rename({'Feature ID':'ASV'}, axis=1).set_index('ASV')

# remove genus tag in species
df_tax['Species'] = [y.replace(x+'-','') for x,y in zip(df_tax['Genus'], df_tax['Species'])]
#df_tax['LowestTaxon'] = [y.replace(x+'-','') for x,y in zip(df_tax['Genus'], df_tax['LowestTaxon'])]

# # select only bacteria
# df_tax = df_tax[df_tax.Kingdom=='Bacteria']

df_tax.head()

,Confidence,Kingdom,Phylum,Class,Order,Family,Genus,Species,LowestClassifiedTaxon
ASV,,,,,,,,,
99deb3c5ecb022ec05609ebd1112a557,0.999997,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,unclassified,Bacteroides
f5f5e0da89730462abaf6301a9557193,0.999580,Bacteria,Firmicutes,Clostridia,Oscillospirales,Ruminococcaceae,Faecalibacterium,unclassified,Faecalibacterium
333bbf9224442fc10cd497377d2b1d01,0.999991,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,unclassified,Bacteroides
263e41ea2c25dc87bd2ff5910cf30d40,0.999999,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,unclassified,unclassified,Lachnospiraceae
b15193fce14759d1c06728933e044af6,0.999991,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,unclassified,Bacteroides


# Read metadata

In [12]:
df_meta = pd.read_csv('metadata-filtered.txt', sep='\t').set_index('#SampleID')
df_meta = df_meta.drop('#q2:types')
df_meta.head()

,forward-absolute-filepath,reverse-absolute-filepath,proband,subject_id,enterotype_jsd,excluded,inulin,test_period,timepoint,responders_bowelfrequency,...,percentage_energy_of_fat,percentage_energy_of_protein,phenotype,physical_discomfort,protein_g,satisfaction,sex,stool_consistency,straining,ENA-CHECKLIST
#SampleID,,,,,,,,,,,,,,,,,,,,,
sample1,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,P01,1,Prev,FALSE,TRUE,inulin,Intake1,responder,...,28,18,"healthy, mildly constipated",0.2,61.76,3.75,f,4,0,ERC000026
sample2,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,P01,1,Prev,FALSE,FALSE,maltodextrin,Intake2,responder,...,42,15,"healthy, mildly constipated",2,66.91,1,f,3,0.5,ERC000026
sample3,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,P01,1,Rum,FALSE,FALSE,run_in-before_inulin,RunIn1,responder,...,36,15,"healthy, mildly constipated",2.2,64.94,1.75,f,3,2.67,ERC000026
sample4,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,P01,1,Rum,FALSE,FALSE,run_in_before_maltodextrine,RunIn2,responder,...,37,14,"healthy, mildly constipated",2.4,69.46,1,f,2,3,ERC000026
sample5,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,/mnt/d/16S_sequencing_data/Vandeputte-2017-Gut...,P03,3,Rum,FALSE,FALSE,maltodextrin,Intake1,responder,...,44,13,"healthy, mildly constipated",2.2,41.09,0,f,2.5,2.67,ERC000026


# Prepare relative abundance table

In [13]:
df_count = pd.read_csv('otu.txt', sep='\t')
df_count = df_count.rename({'#OTU ID':'ASV'}, axis=1).set_index('ASV')
df_count = df_count[list(df_meta.index)]
df_count = df_count.div(df_count.sum(axis=0), axis=1)
df_count = df_count[(df_count.T != 0).any()]

# merge tax and absolute abundance
df_count = pd.merge(df_tax[['LowestClassifiedTaxon']], df_count, left_index=True, right_index=True, how='inner').set_index('LowestClassifiedTaxon', drop=True)

# merge index (row sum for the same index)
df_count = df_count.groupby(df_count.index).sum()

df_count.head()

,sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample8,sample12,sample13,...,sample187,sample188,sample189,sample190,sample191,sample192,sample193,sample194,sample195,sample196
LowestClassifiedTaxon,,,,,,,,,,,,,,,,,,,,,
Acetanaerobacterium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Acidaminococcus-intestini,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Akkermansia,0.012382,0.000000,0.030136,0.003595,0.005820,0.012463,0.006850,0.003931,0.007724,0.011736,...,0.103854,0.017380,0.081228,0.039595,0.011585,0.024691,0.030911,0.000000,0.001977,0.000000
Akkermansia-sp.-KLE1798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018239,0.026983,0.010855
Alistipes,0.018176,0.011868,0.022406,0.045532,0.008059,0.002405,0.006147,0.004392,0.045359,0.052899,...,0.001545,0.013626,0.033497,0.046961,0.029538,0.042278,0.030868,0.011709,0.011985,0.010676


In [14]:
df_count_T = df_count.T
df_count_T.index.name = 'SampleID'

In [15]:
df_count_T.to_excel('16S_relative_abundance.xlsx')